In [28]:
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling import MajorityLabelVoter
from snorkel.labeling import LabelModel
import pandas as pd
import re

In [101]:
df_train = pd.read_csv("r_androiddev data only PII (Elizabeth) - locationset_df.csv") 
df_test = pd.read_csv("r_androiddev data only PII (Elizabeth) - Copy of testset_df.csv")
Y_train = df_train.label.values
Y_test = df_test.label.values
Y_test

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0])

In [40]:
df_test

,Unnamed: 0,keyword,text,label
0,8216,account name,"Congrats!\nIt just takes time to show up, try ...",0
1,5921,account names,You could use the material design guidelines t...,1
2,4680,advertising id,You can check:\n\nhttp://googleadsdeveloper.bl...,1
3,392,audio recording,Stumbled upon this SO question and I took a sw...,1
4,860,camera,My experiences developing on Android:I just fi...,0
5,5001,camera,&gt; The question is why do we need OS updates...,0
6,5084,camera,I don't really care for camera quality but aft...,0
7,1025,camera,Can someone look into this apk and find the en...,0
8,5668,camera,Inspired by Jake Wharton and Jesse Wilson's ta...,0
9,6334,camera,I saw your other comment saying you used Nova ...,0


In [5]:
df_train.dtypes

Unnamed: 0      int64
keyword        object
text           object
label           int64
              float64
dtype: object

In [67]:
ABSTAIN = -1
# HAM = 0
# SPAM = 1
HAM = 1
SPAM = 0

In [173]:
@labeling_function()
def keywords(x):
    return HAM if "fusedlocation" in x.text.lower() or "permission" in x.text.lower() or "google maps" in x.text.lower() or "gps" in x.text.lower() or "location data" in x.text.lower() or "location tracking" in x.text.lower() or "geo" in x.text.lower() or "real-time" in x.text.lower() or "realtime" in x.text.lower() or "fetching" in x.text.lower() or "tracking" in x.text.lower() else ABSTAIN

def keyword_lookup(x, keywords, label):
    if "fusedlocation" in x.text.lower() or "permission" in x.text.lower() or "google maps" in x.text.lower() or "gps" in x.text.lower() or "location data" in x.text.lower() or "location tracking" in x.text.lower() or "geo" in x.text.lower() or "real-time" in x.text.lower() or "realtime" in x.text.lower() or "fetching" in x.text.lower() or "tracking" in x.text.lower(): return HAM
    if any(word in x.text.lower() for word in keywords):
        return label
    return ABSTAIN


def make_keyword_lf(keywords, label=SPAM):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label),
    )

@labeling_function()
def keyword_job(x):
    return SPAM if re.search(r"company.*job|salary|salaries|paid", x.text.lower(), flags=re.S) else ABSTAIN

@labeling_function()
def keyword_directory(x):
    if keywords(x) == HAM: return HAM
    return SPAM if re.search(r"file.*directory", x.text.lower(), flags=re.S) else ABSTAIN

# @labeling_function()
# def keyword_system(x):
#     return SPAM if "system" in x.text.lower() and keywords(x) != ABSTAIN else ABSTAIN

keyword_system = make_keyword_lf(keywords = ["system"])

@labeling_function()
def keyword_file_loc(x):
    return SPAM if "file location" in x.text.lower() or re.search(r"location of.*files", x.text.lower(), flags=re.S) else ABSTAIN

# UI covers a lot
keyword_UI = make_keyword_lf(keywords = [" ui"])
# @labeling_function()
# def keyword_UI(x):
#     return SPAM if " ui" in x.text.lower() else ABSTAIN

keyword_git = make_keyword_lf(keywords = ["git"])


In [174]:
lfs = [keywords, keyword_job, keyword_directory, keyword_system, keyword_file_loc, keyword_UI, keyword_git]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

100%|██████████| 200/200 [00:00<00:00, 1490.54it/s]


In [196]:
L_train 

array([[ 1, -1,  1,  1, -1,  1,  1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1,  0, -1, -1,  0],
       [-1, -1, -1, -1, -1,  0, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [ 1, -1,  1,  1, -1,  1,  1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1,  0],
       [-1, -1,  0, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1,  0],
       [-1, -1, -1, -1,  0, -1, -1],
       [-1, -1, -1,  0, -1,  0,  0],
       [-1, -1, -1, -1, -1,  0, -1],
       [ 1, -1,  1,  1, -1,  1,  1],
       [ 1,  0,  1,  1, -1,  1,  1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1,  0],
       [-1, -1, -1, -1,  0, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1],
       [-1,  0, -1, -1, -1, -1, -1],
       [ 1, -1,  1,  1, -1,  1,  1],
       [-1, -1, -1, -1, -1, -1, -1],
       [ 1, -1,  1,  1, -1,  1,  1],
       [ 1, -1,  1,  1, -1,  1,  1],
       [-1, -1, -1, -1, -1, -1, -1],
       [ 1, -1,  1,  1, -1,  1,  1],
 

In [176]:
coverage_keywords, coverage_job, coverage_directory, coverage_system, coverage_file_loc, coverage_UI, coverage_git = (L_train != ABSTAIN).mean(axis=0)
print(f"keywords coverage: {coverage_keywords * 100:.1f}%")
print(f"job coverage: {coverage_job * 100:.1f}%")
print(f"directory coverage: {coverage_directory * 100:.1f}%")
print(f"system coverage: {coverage_system * 100:.1f}%")
print(f"file location coverage: {coverage_file_loc * 100:.1f}%")
print(f"UI coverage: {coverage_UI * 100:.1f}%")
print(f"git coverage: {coverage_git * 100:.1f}%")

keywords coverage: 53.3%
job coverage: 4.0%
directory coverage: 54.7%
system coverage: 56.0%
file location coverage: 2.7%
UI coverage: 57.3%
git coverage: 61.3%


In [177]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
keywords,0,[1],0.533333,0.533333,0.013333
keyword_job,1,[0],0.040000,0.013333,0.013333
keyword_directory,2,"[0, 1]",0.546667,0.533333,0.013333
keyword_system,3,"[0, 1]",0.560000,0.560000,0.013333
keyword_file_loc,4,[0],0.026667,0.000000,0.000000
keyword_ ui,5,"[0, 1]",0.573333,0.546667,0.013333
keyword_git,6,"[0, 1]",0.613333,0.560000,0.013333


In [178]:
df_train.iloc[L_train[:, 0] == HAM].sample(frac=10, replace=True)

,Unnamed: 0,keyword,text,label,
35,1464,location,Accurate Location Updates?:So i want to track ...,1,NaN
73,10640,locations,"&gt; There is no such thing as "" Impregnable"" ...",1,NaN
47,1943,location,Background processing:I am developing an app a...,1,NaN
42,7952,location,"Hello, I have a Geotag Notes app I am working ...",1,NaN
39,7105,location,Hmm...strange library design. You might want t...,1,NaN
55,11634,location,"Hi!\n\n&amp;#x200B;\n\nIn the meanwhile, I did...",1,NaN
21,3438,location,So you want your app X to work with google map...,1,NaN
26,636,location,Trouble querying database with GPS coordinates...,1,NaN
37,7022,location,There's no way to do this alone on the device ...,1,NaN
27,653,location,Looking to list nearest locations based off of...,1,NaN


In [179]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
keywords,0,[1],0.250,0.250,0.01
keyword_job,1,[0],0.040,0.020,0.01
keyword_directory,2,"[0, 1]",0.265,0.255,0.01
keyword_system,3,"[0, 1]",0.300,0.275,0.01
keyword_file_loc,4,[0],0.010,0.000,0.00
keyword_ ui,5,"[0, 1]",0.275,0.260,0.01
keyword_git,6,"[0, 1]",0.305,0.265,0.01


In [180]:
df_test.iloc[L_test[:, 1] == SPAM]

,Unnamed: 0,keyword,text,label
40,1945,camera,Camera preview 1 is not full size/stretches:He...,1
45,12299,cameras,I think Google's APIs are dramatically more de...,0
66,853,home address,Disagree with Google's new publicly-available ...,0
93,11094,location,*Company*: Freetrade\n\n*Job*: Senior Android ...,0
98,3178,location,Very good points. A lot of people forget about...,0
147,11789,locations,Imagine that things used to be even worse hah....,0
189,9047,voice,Company: Discord\n\nJob: Senior Android Develo...,0
192,11470,voice,Company: &lt;Play Sports Network&gt;\n\nJob: &...,0


In [181]:
majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)
preds_train

array([1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1])

In [182]:
preds_test = majority_model.predict(L=L_test)
preds_test

array([0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1])

In [183]:
# Recall: true positives/all positives
# Specificity: true negatives/all negatives

def stats(act_labels, pred_labels):
    true_pos, false_pos, true_neg, false_neg = 0, 0, 0, 0
    for i in range(len(act_labels)):
        if pred_labels[i] == 0:
            if act_labels[i] == 0:
                true_neg += 1
            elif act_labels[i] == 1:
                false_pos += 1
        elif pred_labels[i] == 1:
            if act_labels[i] == 0:
                false_neg += 1
            elif act_labels[i] == 1:
                true_pos += 1
    return true_pos, false_pos, true_neg, false_neg

In [184]:
true_pos_train, false_pos_train, true_neg_train, false_neg_train = stats(Y_train, preds_train)
recall_train = true_pos_train/(true_pos_train + false_neg_train)
specificity_train = true_neg_train/(true_neg_train + false_pos_train)

In [185]:
true_pos_test, false_pos_test, true_neg_test, false_neg_test = stats(Y_test, preds_test)
recall_test = true_pos_test/(true_pos_test + false_neg_test)
specificity_test = true_neg_test/(true_neg_test + false_pos_test)

In [193]:
majority_acc_train = majority_model.score(L=L_train, Y=Y_train, tie_break_policy="random")[
    "accuracy"
]
majority_acc_test = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]

In [192]:
print(f"{'Majority Vote Accuracy (Train):':<25} {majority_acc_train * 100:.1f}%")
print(f"{'Recall (Train):':<25} {recall_train * 100:.1f}%")
print(f"{'Specificity (Train):':<25} {specificity_train * 100:.1f}%")

Majority Vote Accuracy (Train): 82.7%
Recall (Train):           83.6%
Specificity (Train):      80.0%


In [194]:
print(f"{'Majority Vote Accuracy (Test):':<25} {majority_acc_test * 100:.1f}%")
print(f"{'Recall (Test):':<25} {recall_test * 100:.1f}%")
print(f"{'Specificity (Test):':<25} {specificity_test * 100:.1f}%")

Majority Vote Accuracy (Test): 62.0%
Recall (Test):            69.6%
Specificity (Test):       51.8%
